# DSCI 525: Milestone 1

## Installing packages that are not included in the environment file

In [14]:
import sys
#!conda install --yes --prefix {sys.prefix} altair

## Library Imports

In [12]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage
import dask.dataframe as dd
import gc
import sys
import dask
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
import altair as alt
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import dask.dataframe as dd
import pyarrow.feather as feather

In [15]:
%load_ext rpy2.ipython
%load_ext memory_profiler

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


## 1. Team-work contract

- Please find the link to the teamwork contract [here](https://docs.google.com/document/d/1Sfs2RBib0FiGaDob7pGoitGPthRL-F32-dZo7A-aOhg/edit#heading=h.tgeu41bizsao).

## 2. Creating repository and project structure

- Please find the GitHub repository for the project [here](https://github.com/UBC-MDS/525_group18).

## 3. Downloading the Data

In [17]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshare_rainfall/"

Get the necessary files from the figshareAPI for the rainfall article.

In [18]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  
files = data["files"]             
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

We are interested to download the `data.zip` file from the rainfall article from figshare.

We are also timing how long it takes to download the data through the figshareAPI

In [6]:
%%time
files_to_dl = ["data.zip"]  
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 8.54 s, sys: 6.98 s, total: 15.5 s
Wall time: 2min 11s


Extracting the file from the `data.zip` folder

In [19]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

Wall time: 21.6 s


## 4. Comibining the data CSVs with Pandas and Dask

### 4.1 Using Pandas

In [8]:
%%time
%memit

com_dat_path = "figshare_rainfall/combined_data.csv"

files = glob.glob('figshare_rainfall/*NSW.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall(r'[^\/]+(?=\_d)', file)[0])
                for file in files)
              )
df.to_csv(com_dat_path)

peak memory: 192.93 MiB, increment: 0.03 MiB
CPU times: user 7min 4s, sys: 13.5 s, total: 7min 18s
Wall time: 7min 59s


In [9]:
%%sh
du -sh figshare_rainfall/combined_data.csv

5.6G	figshare_rainfall/combined_data.csv


In [10]:
%%time
df = pd.read_csv("figshare_rainfall/combined_data.csv")

CPU times: user 1min 16s, sys: 22.4 s, total: 1min 38s
Wall time: 6min 4s


In [11]:
print(f"The data Usage for combined dataframe in pandas is {df.memory_usage(index=True).sum()}")

The data Usage for combined dataframe in pandas is 3498199336


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62467843 entries, 0 to 62467842
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           object 
 1   lat_min        float64
 2   lat_max        float64
 3   lon_min        float64
 4   lon_max        float64
 5   rain (mm/day)  float64
 6   model          object 
dtypes: float64(5), object(2)
memory usage: 3.3+ GB


In [13]:
sys.getsizeof(df)

11448609766

In [14]:
print(df.shape)

(62467843, 7)


In [15]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.811518,-34.86911,140.625,141.875,3.045650e-13,SAM0-UNICON
1,1889-01-02 12:00:00,-35.811518,-34.86911,140.625,141.875,3.572392e-04,SAM0-UNICON
2,1889-01-03 12:00:00,-35.811518,-34.86911,140.625,141.875,9.431562e-03,SAM0-UNICON
3,1889-01-04 12:00:00,-35.811518,-34.86911,140.625,141.875,9.663865e-02,SAM0-UNICON
4,1889-01-05 12:00:00,-35.811518,-34.86911,140.625,141.875,0.000000e+00,SAM0-UNICON


***Observations***

- Zhenrui：

    - Combining files:
        - peak memory: 2770.49 MiB, increment: 0.02 MiB
        - Wall time: 6min 56s
    - Reading combined file:
        - Wall time: 1min 03s
    - Combined Csv size
        - 5.7G - figshare_rainfall/combined_data.csv
    - System resource Usage:
        - Ram: 5.8 GB
        - Cache: 2.9 GB

- Bruhat:
    - Combining files:
        - peak memory: 154.38 MiB, increment: 0.00 MiB
        - Wall time: 7min 38s
    - Reading combined file:
        - Wall time: 4min 6s
    - Combined Csv size
        - 5.6G - figshare_rainfall/combined_data.csv
    - System resource Usage:
        - Ram: 5.7 GB
        - Cache: 2.9 GB

- Deepak: Panda's way of combining the different csv files did not work for me, my computer crushed while processing the code. Since I only have 4GB RAM available for memory.
    

### Using Dask

In [16]:

%%time
%memit
file_list = glob.glob('figshare_rainfall/*NSW.csv')

def read_and_label_csv(filename):
    '''
    This function loads the csv file and adds a 'Model' column
    based on the filename
    
    Parameters
    ----------
    filename: str
        relative path to the file 
    
    Returns
    -------
        df_csv, a dataframe
    '''

    # reads each csv file to a pandas.DataFrame
    df_csv = pd.read_csv(filename, index_col=0)
    df_csv['Model'] = re.findall(r'[^\/]+(?=\_d)',filename)[0]
    return df_csv 

# create a list of functions ready to return a DataFrame
dfs = [dask.delayed(read_and_label_csv)(fname) for fname in file_list]

# using delayed, assemble the pandas.DataFrames into a dask.DataFrame
ddf = dd.from_delayed(dfs)

ddf.to_csv("figshare_rainfall/combined_data_dask.csv", single_file=True)

peak memory: 5328.51 MiB, increment: 0.84 MiB
CPU times: user 7min 54s, sys: 18.7 s, total: 8min 12s
Wall time: 7min 20s


['/home/bruhat/april_block/525_web_cloud_comp/525_group18/notebooks/figshare_rainfall/combined_data_dask.csv']

In [17]:
%%sh
du -sh figshare_rainfall/combined_data.csv

5.6G	figshare_rainfall/combined_data.csv


In [18]:
%%time
df = dd.read_csv("figshare_rainfall/combined_data.csv")

CPU times: user 25.7 ms, sys: 78 µs, total: 25.8 ms
Wall time: 53.9 ms


In [19]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.811518,-34.86911,140.625,141.875,3.045650e-13,SAM0-UNICON
1,1889-01-02 12:00:00,-35.811518,-34.86911,140.625,141.875,3.572392e-04,SAM0-UNICON
2,1889-01-03 12:00:00,-35.811518,-34.86911,140.625,141.875,9.431562e-03,SAM0-UNICON
3,1889-01-04 12:00:00,-35.811518,-34.86911,140.625,141.875,9.663865e-02,SAM0-UNICON
4,1889-01-05 12:00:00,-35.811518,-34.86911,140.625,141.875,0.000000e+00,SAM0-UNICON


In [20]:
df.dtypes

time              object
lat_min          float64
lat_max          float64
lon_min          float64
lon_max          float64
rain (mm/day)    float64
model             object
dtype: object

***Observations***

- Deepak：
    - Combining files:
        - peak memory: 175.45 MiB, increment: 0.37 MiB
        - Wall time: 7min 44s
    - Reading combined file:
        - Wall time: 444 ms
    - Combined Csv size
        - 6.0G - figshare_rainfall/combined_data_dask.csv
    - System resource Usage:
        - Ram: 4.5 GB
        - Cache: 1.5 GB

- Zhenrui：
    - Combining files:
        - peak memory: 174.59 MiB, increment: 0.00 MiB
        - Wall time: 7min 50s
    - Reading combined file:
        - Wall time: 22.7 ms
    - Combined Csv size
        - 6.7G - figshare_rainfall/combined_data_dask.csv
    - System resource Usage:
        - Ram: 4.5 GB
        - Cache: 1.5 GB   
        
- Bruhat：
    - Combining files:
        - peak memory: 5328.51 MiB, increment: 0.84 MiB
        - Wall time: 7min 20s
    - Reading combined file:
        - Wall time: 53.9 ms
    - Combined Csv size
        - 5.6G	figshare_rainfall/combined_data.csv
    - System resource Usage:
        - Ram: 5.5 GB
        - Cache: 2.5 GB   

By comparision, dask obviosly has the faster read speed.

## 5. Load the combined CSV to memory and perform a simple EDA

In [21]:
#Clearing out previous memory 
del df
gc.collect()

498

### 5.1 Dask Approach to reduce memory usage

In [7]:
%%time
%memit
df_dask = dd.read_csv(com_dat_path)

peak memory: 190.88 MiB, increment: 0.02 MiB
CPU times: user 60 ms, sys: 64.5 ms, total: 124 ms
Wall time: 278 ms


In [8]:
df_dask.memory_usage(deep=True)

Dask Series Structure:
npartitions=1
    int64
      ...
dtype: int64
Dask Name: series-groupby-sum-agg, 391 tasks

In [9]:
print(df_dask.model.value_counts().compute())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64


In [11]:
df_dask.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 7 entries, time to model
dtypes: object(2), float64(5)

### 5.2 Loading just columns what we want approach to reduce memory usage.

In [21]:
%%time
%memit

#loading time and rain col to perform eda
df_reqcol = pd.read_csv(com_dat_path, usecols = ['time', 'rain (mm/day)'])
df_reqcol['time'] = pd.to_datetime(df_reqcol['time'])

peak memory: 8849.23 MiB, increment: 0.00 MiB
Wall time: 1min 36s


In [22]:
df_reqcol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62467843 entries, 0 to 62467842
Data columns (total 2 columns):
 #   Column         Dtype         
---  ------         -----         
 0   time           datetime64[ns]
 1   rain (mm/day)  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 953.2 MB


- By using dask, due to the feature of multiprocessing or parrallel, it took much less time to read the data. Reading with pandas took 6min 4s where as in dask, it was only 278 ms. Initially, we tried reading the data and Changing dtype of the data. That did not give us any significant improvement in space saving.

- The peak memory with dask was 190.88 MiB which was considerably lower than reading with pandas.

- Loading in chunks also seemed like a viable option, but in practice turned out to increase complexity of code without significant increase in memory savings compared to dask.

- If we are trying to stray away from dask, A viable alternative is to just load the columns that we need and perform the eda, which is an alternative we tried above. The memory usage was 953.2 MB which is far better than the size of whole data frame (5.6Gb).

## 6. Perform a simple EDA in R

### 6.1 Using Pandas

In [6]:
%%time
%%memit
#simple pandas - This is how we do normally ,which means we are loading the entire data to the memory
# just loading 10 million rows
df = pd.read_csv("figshare_rainfall/combined_data.csv")

peak memory: 6978.16 MiB, increment: 6631.93 MiB
CPU times: user 1min 8s, sys: 15.3 s, total: 1min 24s
Wall time: 3min 47s


In [ ]:
# %%time
# %%R -i df

# start_time <- Sys.time()
# library(dplyr)
# result <- df %>% count(model)
# end_time <- Sys.time()
# print(end_time - start_time)

***does not run in our computers (ran it for 2hrs with 18gb ram and it eventually just crashes)***

### 6.2 Arrow + parquet

In [4]:
%%time
%%memit
# Read data file and prepare arrow table
dataset = ds.dataset("figshare_rainfall/combined_data.csv", format="csv")
table = dataset.to_table()

peak memory: 4303.03 MiB, increment: 4112.61 MiB
CPU times: user 29.1 s, sys: 3.77 s, total: 32.9 s
Wall time: 30.4 s


In [5]:
%%time
pq.write_to_dataset(table, 'figshare_rainfall/combined_data.parquet')

CPU times: user 10 s, sys: 461 ms, total: 10.5 s
Wall time: 10.9 s


In [6]:
%%sh
du -sh figshare_rainfall/combined_data.csv
du -sh figshare_rainfall/combined_data.parquet

5.6G	figshare_rainfall/combined_data.csv
1.1G	figshare_rainfall/combined_data.parquet


In [20]:
%%time
%%R

library(arrow)
start_time <- Sys.time()
file_name <- list.files(path = "figshare_rainfall/combined_data.parquet/", pattern="\\.parquet$")
r_table <- arrow::read_parquet(paste("figshare_rainfall/combined_data.parquet/", file_name, sep=""))
print(class(r_table))
library(dplyr)
result <- r_table %>% count(model)
end_time <- Sys.time()
print(result)
print(end_time - start_time)

R[write to console]: 
Attaching package: ‘arrow’


R[write to console]: The following object is masked from ‘package:utils’:

    timestamp




[1] "tbl_df"     "tbl"        "data.frame"


R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# A tibble: 27 x 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# … with 17 more rows
Time difference of 56.10742 secs
CPU times: user 11.7 s, sys: 7.96 s, total: 19.7 s
Wall time: 57.4 s


### 6.3 Arrow + feather

In [8]:
%%time
feather.write_feather(table, 'figshare_rainfall/combined_data.feather')

CPU times: user 5.56 s, sys: 1.54 s, total: 7.1 s
Wall time: 9.81 s


In [9]:
%%sh
du -sh figshare_rainfall/combined_data.feather

1.1G	figshare_rainfall/combined_data.feather


In [10]:
%%time
%%R

library(arrow)
start_time <- Sys.time()
r_table <- arrow::read_feather("figshare_rainfall/combined_data.feather")
print(class(r_table))
library(dplyr)
result <- r_table %>% count(model)
end_time <- Sys.time()
print(result)
print(end_time - start_time)

[1] "tbl_df"     "tbl"        "data.frame"


R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# A tibble: 27 x 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# … with 17 more rows
Time difference of 3.334692 mins
CPU times: user 16 s, sys: 15.4 s, total: 31.4 s
Wall time: 3min 20s


Comparing the storage usage and time spending, arrow and parquet is the best option. Obviously, transfering the python dataframe to R is not an ideal way for the data with quite big size. We waited for over 20 minuties, there is still no result. Compared with the `csv` file, the feather and parquet both have the smaller size. However, our group found that it seems Windows cannot use feather file with arrow table. Normally, it only takes around 3 minutes to count the model. But the Windows cannot run the cell sccuessfully. On the contrary, parquet file with arrow table is the best option, which takes only 57.4s to count the model, and it can run properly both in Windows and Linux (we do not have Mac user in our group).